In [ ]:
# Read data from google drive 
from google.colab import drive
drive.mount('/content/drive')

import os
# path = '/content/drive/MyDrive/projet_recherche'
path = '/content/drive/MyDrive/ENSIIE/M1/S4/PRR-Projet-Recherche/projet_recherche'
os.chdir(path)

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import datetime
from functions import MAPE, MSE, MAE

from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv("data/param2.csv")

In [ ]:
df.shape

(100000, 12)

In [ ]:
df.head()

,S,K,m,T,r,rho,kappa,gam,nu,V0,price,impl_vol
0,100.0,109.727579,0.911348,1.289351,0.012616,-0.341331,0.466980,0.570688,0.037302,0.470395,22.052446,0.557978
1,100.0,121.761607,0.821277,1.748494,0.003466,-0.184956,0.325467,0.269769,0.112852,0.493984,25.489026,0.619066
2,100.0,75.938311,1.316858,0.718012,0.015625,-0.786272,1.965432,0.464774,0.181610,0.135622,28.586334,0.421757
3,100.0,75.997247,1.315837,1.180228,0.041085,-0.777110,1.047442,0.168897,0.027497,0.346401,34.370636,0.468116
4,100.0,101.204579,0.988098,0.699343,0.021379,-0.694197,0.622226,0.649895,0.468941,0.066212,11.764919,0.349931


# Forward Pass

### Model predicting the price

This first model predicts the price given the parameters of the Heston model. There are still a few things to check, but as a first approach it's looking good.  

In [ ]:
X = df.drop(['K', 'S', 'price', 'impl_vol'], axis = 1)
# X_stand =(X-X.mean())/X.std()
y = df['price']
# y_norm = (y-y.mean())/y.std()

In [ ]:
X.shape

(100000, 8)

In [ ]:
# The dataset should be split into 10-validation, 10-testing, 80-training 
# However, to keep thing simple, we use just a 20-80 test-train split
# X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [ ]:
train_ratio = 0.8
validation_ratio = 0.1
test_ratio = 0.1

# train is now 80% of the entire data set
# the _junk suffix means that we drop that variable completely
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=1 - train_ratio, random_state=1)

# test is now 10% of the initial data set
# validation is now 10% of the initial data set
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio), random_state=1)

In [ ]:
# epochs is the number of iterations over the data
# verbose is a parameter to obtain additional information 
# Note: define the batch size to 
# Check if they use fully conected layers (dense) on the paper
initializer = tf.keras.initializers.GlorotUniform()
model1 = tf.keras.Sequential([
    keras.layers.Dense(units=200,input_shape=(8,),  kernel_initializer=initializer, activation='relu'),
    keras.layers.Dense(units=200,activation='relu'),
    keras.layers.Dense(units=200,activation='relu'),
    keras.layers.Dense(units=200,activation='relu'),
    keras.layers.Dense(units=1),
    ])
model1.compile(optimizer='adam', 
               loss='mean_squared_error',
               metrics =[keras.metrics.MeanSquaredError(),
                         keras.metrics.MeanAbsoluteError(),
                         keras.metrics.MeanAbsolutePercentageError()])
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 200)               1800      
                                                                 
 dense_1 (Dense)             (None, 200)               40200     
                                                                 
 dense_2 (Dense)             (None, 200)               40200     
                                                                 
 dense_3 (Dense)             (None, 200)               40200     
                                                                 
 dense_4 (Dense)             (None, 1)                 201       
                                                                 
Total params: 122,601
Trainable params: 122,601
Non-trainable params: 0
_________________________________________________________________


In [ ]:
start = datetime.datetime.now()
print("training start time:", start)
history1= model1.fit(x_train, y_train, epochs=8000, batch_size= 1024, validation_data=(x_val, y_val), verbose=0)
end = datetime.datetime.now()
print("training end time:", end)
print('total time:', end-start)

In [ ]:
hist1 = history1.history

In [ ]:
for key in hist1.keys() :
  print(key, ':', hist1[key][8000-1])

loss : 0.001518822624348104
mean_squared_error : 0.001518822624348104
mean_absolute_error : 0.030641833320260048
mean_absolute_percentage_error : 10833.2236328125
val_loss : 0.0054627154022455215
val_mean_squared_error : 0.0054627154022455215
val_mean_absolute_error : 0.057078681886196136
val_mean_absolute_percentage_error : 26495.01953125


In [ ]:
results1 = model1.evaluate(x_test, y_test)

79/79 [==============================] - 0s 3ms/step - loss: 0.0054 - mean_squared_error: 0.0054 - mean_absolute_error: 0.0567 - mean_absolute_percentage_error: 18037.8906


In [ ]:
for i in range(len(list(hist1.keys())[0:4])) :
  print(list(hist1.keys())[i], ':', results1[i])

loss : 0.005353164859116077
mean_squared_error : 0.005353164859116077
mean_absolute_error : 0.056735455989837646
mean_absolute_percentage_error : 18037.890625


In [ ]:
y_pred = model1.predict(x_test)

In [ ]:
y_test.head(10)

82381    42.380078
46057    16.905663
74567     1.493860
5282     29.524343
86634    43.531872
17513    29.435774
34099    27.826019
86343    43.963627
93382    20.749449
68808     1.675176
Name: price, dtype: float64

In [ ]:
y_pred[0:10]

array([[42.344017 ],
       [16.908045 ],
       [ 1.4663668],
       [29.517649 ],
       [43.42189  ],
       [29.395102 ],
       [27.795198 ],
       [43.892387 ],
       [20.660273 ],
       [ 1.6506165]], dtype=float32)

In [ ]:
MSE(y_test, y_pred)

2765960.05045

In [ ]:
MAE(y_test, y_pred)

135180.88815

In [ ]:
MAPE(y_test, y_pred)

5730024471158427.0

In [ ]:
file_name = 'model1'
pd.DataFrame(hist1).to_csv('results/'+ file_name + '_train.csv', index = False)

In [ ]:
pd.DataFrame(results1).transpose().set_axis(['loss', 'mean_squared_error', 'mean_absolute_error', 'mean_absolute_percentage_error'], axis=1).to_csv('results/'+ file_name +'_test.csv', index = False)

### Model predicting the implied volatility

In [ ]:
X = df.drop(['price', 'K', 'S', 'impl_vol'], axis=1)
# X_stand =(X-X.mean())/X.std()
y = df['impl_vol']
# y_norm = (y-y.mean())/y.std()

In [ ]:
X.shape

(100000, 8)

In [ ]:
# The dataset should be split into 10-validation, 10-testing, 80-training 
# X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [ ]:
train_ratio = 0.8
validation_ratio = 0.1
test_ratio = 0.1

# train is now 80% of the entire data set
# the _junk suffix means that we drop that variable completely
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=1 - train_ratio, random_state=1)

# test is now 10% of the initial data set
# validation is now 10% of the initial data set
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio), random_state=1)

In [ ]:
# epochs is the number of iterations over the data
# verbose is a parameter to obtain additional information 
# Note: define the batch size to 
initializer = tf.keras.initializers.GlorotUniform()
model2 = tf.keras.Sequential([
    keras.layers.Dense(units=200,input_shape=(8,),  kernel_initializer=initializer, activation='relu'),
    keras.layers.Dense(units=200,activation='relu'),
    keras.layers.Dense(units=200,activation='relu'),
    keras.layers.Dense(units=200,activation='relu'),
    keras.layers.Dense(units=1),
    ])
model2.compile(optimizer='adam', 
               loss='mean_squared_error',
               metrics =[keras.metrics.MeanSquaredError(),
                         keras.metrics.MeanAbsoluteError(),
                         keras.metrics.MeanAbsolutePercentageError()])
model2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 200)               1800      
                                                                 
 dense_1 (Dense)             (None, 200)               40200     
                                                                 
 dense_2 (Dense)             (None, 200)               40200     
                                                                 
 dense_3 (Dense)             (None, 200)               40200     
                                                                 
 dense_4 (Dense)             (None, 1)                 201       
                                                                 
Total params: 122,601
Trainable params: 122,601
Non-trainable params: 0
_________________________________________________________________


In [ ]:
start = datetime.datetime.now()
print("training start time:", start)
history2= model2.fit(x_train, y_train, epochs=8000, batch_size= 1024, validation_data=(x_val, y_val), verbose=0)
end = datetime.datetime.now()
print("training end time:", end)
print('total time:', end-start)

training start time: 2022-04-23 15:06:02.316187
training end time: 2022-04-23 16:05:26.026481
total time: 0:59:23.710294


In [ ]:
hist2 = history2.history

In [ ]:
for key in hist2.keys() :
  print(key, ':', hist2[key][8000-1])

loss : 2.538328089940478e-07
mean_squared_error : 2.538328089940478e-07
mean_absolute_error : 0.0004012206627521664
mean_absolute_percentage_error : 101.51593017578125
val_loss : 4.0691334106668364e-06
val_mean_squared_error : 4.0691334106668364e-06
val_mean_absolute_error : 0.0007561574457213283
val_mean_absolute_percentage_error : 3536.4375


In [ ]:
file_name = 'model2'
pd.DataFrame(hist2).to_csv('results/'+ file_name + '_train.csv', index = False)

In [ ]:
results2 = model2.evaluate(x_test, y_test)

313/313 [==============================] - 1s 4ms/step - loss: 1.9140e-06 - mean_squared_error: 1.9140e-06 - mean_absolute_error: 7.4602e-04 - mean_absolute_percentage_error: 0.2086


In [ ]:
for i in range(len(list(hist2.keys())[0:4])) :
  print(list(hist2.keys())[i], ':', results2[i])

loss : 1.914025233418215e-06
mean_squared_error : 1.914025233418215e-06
mean_absolute_error : 0.0007460200577042997
mean_absolute_percentage_error : 0.20858025550842285


In [ ]:
pd.DataFrame(results2).transpose().set_axis(['loss', 'mean_squared_error', 'mean_absolute_error', 'mean_absolute_percentage_error'], axis=1).to_csv('results/'+ file_name +'_test.csv', index = False)

In [ ]:
# y_pred = model.predict(x_test)
y_pred = model2.predict(x_test)

In [ ]:
y_test.head()

82381    0.612825
46057    0.566543
74567    0.283900
5282     0.652536
86634    0.566911
Name: impl_vol, dtype: float64

In [ ]:
y_pred[0:5]

array([[0.61214507],
       [0.5652702 ],
       [0.28207326],
       [0.65184397],
       [0.5654832 ]], dtype=float32)

In [ ]:
MAPE(y_test, y_pred)

3453.58047

In [ ]:
MSE(y_test, y_pred)

290.25779

In [ ]:
MAE(y_test, y_pred)

1381.44005

# Backward Pass

In [ ]:
df.head()

,m,T,r,rho,kappa,gamma,nu,nu0,K,S,price,impl_vol
0,1.011627,2.151532,0.008438,-0.768919,0.604602,0.374461,0.031367,0.163479,98.850621,100,18.197765,0.291737
1,0.696514,0.465436,0.015462,-0.311999,0.426748,0.642626,0.133013,0.106081,143.572211,100,0.494581,0.310906
2,0.962829,0.308108,0.012195,-0.336677,0.706375,0.523697,0.496252,0.152804,103.860574,100,7.834435,0.421304
3,0.775495,0.999134,0.014837,-0.465915,1.917854,0.421338,0.154169,0.487887,128.949875,100,12.253690,0.523649
4,0.801679,1.663048,0.030446,-0.158940,2.168413,0.631225,0.056295,0.444986,124.738234,100,13.240815,0.382714


In [ ]:
# X = df.drop(['K', 'S', 'price', 'impl_vol'], axis = 1)
X = df[['m', 'T', 'r', 'price']]
y = df[['rho',	'kappa',	'gamma',	'nu',	'nu0']]

In [ ]:
X.shape

(50, 4)

In [ ]:
train_ratio = 0.8
validation_ratio = 0.1
test_ratio = 0.1

# train is now 80% of the entire data set
# the _junk suffix means that we drop that variable completely
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=1 - train_ratio, random_state=1)

# test is now 10% of the initial data set
# validation is now 10% of the initial data set
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio), random_state=1)

In [ ]:
# epochs is the number of iterations over the data
# verbose is a parameter to obtain additional information 
# Note: define the batch size to 
initializer = tf.keras.initializers.GlorotUniform()
model3 = tf.keras.Sequential([
    keras.layers.Dense(units=200,input_shape=(4,),  kernel_initializer=initializer, activation='relu'),
    keras.layers.Dense(units=200,activation='relu'),
    keras.layers.Dense(units=200,activation='relu'),
    keras.layers.Dense(units=200,activation='relu'),
    keras.layers.Dense(units=5),
    ])
model3.compile(optimizer='adam', 
               loss='mean_squared_error',
               metrics =[keras.metrics.MeanSquaredError(),
                         keras.metrics.MeanAbsoluteError(),
                         keras.metrics.MeanAbsolutePercentageError()])
model3.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_40 (Dense)            (None, 200)               1000      
                                                                 
 dense_41 (Dense)            (None, 200)               40200     
                                                                 
 dense_42 (Dense)            (None, 200)               40200     
                                                                 
 dense_43 (Dense)            (None, 200)               40200     
                                                                 
 dense_44 (Dense)            (None, 5)                 1005      
                                                                 
Total params: 122,605
Trainable params: 122,605
Non-trainable params: 0
_________________________________________________________________


In [ ]:
start = datetime.datetime.now()
print("training start time:", start)
history3 = model3.fit(x_train, y_train, epochs=8000, batch_size= 1024, validation_data=(x_val, y_val), verbose=0)
end = datetime.datetime.now()
print("training end time:", end)
print('total time:', end-start)

In [ ]:
hist3 = history3.history

In [ ]:
for key in hist3.keys() :
  print(key, ':', hist3[key][8000-1])

loss : 3.6670175177278e-05
mean_squared_error : 3.6670175177278e-05
mean_absolute_error : 0.0034309164620935917
mean_absolute_percentage_error : 1.6339813470840454
val_loss : 0.16726723313331604
val_mean_squared_error : 0.16726723313331604
val_mean_absolute_error : 0.28373950719833374
val_mean_absolute_percentage_error : 145.661865234375


In [ ]:
file_name = 'model3'
pd.DataFrame(hist3).to_csv('results/'+ file_name + '_train.csv', index = False)

In [ ]:
results3 = model3.evaluate(x_test, y_test)

1/1 [==============================] - 0s 26ms/step - loss: 0.6011 - mean_squared_error: 0.6011 - mean_absolute_error: 0.4482 - mean_absolute_percentage_error: 255.6720


In [ ]:
for i in range(len(list(hist3.keys())[0:4])) :
  print(list(hist3.keys())[i], ':', results3[i])

loss : 0.601050078868866
mean_squared_error : 0.601050078868866
mean_absolute_error : 0.44820624589920044
mean_absolute_percentage_error : 255.6719970703125


In [ ]:
pd.DataFrame(results3).transpose().set_axis(['loss', 'mean_squared_error', 'mean_absolute_error', 'mean_absolute_percentage_error'], axis=1).to_csv('results/'+ file_name +'_test.csv', index = False)

In [ ]:
y_pred = model3.predict(x_test)

In [ ]:
y_pred[0:5]

array([[-0.6601878 ,  0.4465136 ,  0.28418392, -0.0231315 ,  0.19249883],
       [-0.9774316 ,  3.2769225 ,  0.68463176,  0.19006996,  0.36382124],
       [-0.48925346,  1.3867248 ,  0.44393486,  0.21868701,  0.24605778],
       [-0.80881125,  1.9439543 ,  0.45355934,  0.16997072,  0.19601882],
       [-0.43358386,  1.4831842 ,  0.33350527,  0.11203669,  0.23243427]],
      dtype=float32)

In [ ]:
y_test

,rho,kappa,gamma,nu,nu0
40,-0.481483,0.172269,0.705185,0.390793,0.176374
31,-0.216660,0.116518,0.281279,0.080974,0.198034
48,-0.035668,0.325623,0.617970,0.194191,0.095574
2,-0.336677,0.706375,0.523697,0.496252,0.152804
27,-0.672632,0.823755,0.510677,0.038782,0.372226
